<a href="https://colab.research.google.com/github/KhangTheKangaroo/Image-Retrieval/blob/main/Image_Retrieval_with_CLIP_(No_vector_database).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install chromadb
!pip install open-clip-torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 472.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 50.8 MB/s eta 0:00:0

In [2]:
!gdown --id 1msLVo0g0LFmL9-qZ73vq9YEVZwbzOePF # Download the dataset
!unzip -q data.zip

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1msLVo0g0LFmL9-qZ73vq9YEVZwbzOePF
From (redirected): https://drive.google.com/uc?id=1msLVo0g0LFmL9-qZ73vq9YEVZwbzOePF&confirm=t&uuid=121bd718-c637-4b9b-bc52-21ae497338b4
To: /content/data.zip
100% 76.1M/76.1M [00:00<00:00, 112MB/s]


In [3]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction

In [ ]:
def plot_results(query_path, ls_path_score, reverse=False):
    ls_path_score.sort(key=lambda x: x[1], reverse=reverse)  # Sort by score

    # Display query image
    query_image = plt.imread(query_path)
    plt.figure(figsize=(5, 5))
    plt.imshow(query_image)
    plt.title("Query Image")
    plt.axis('off')
    plt.show()

    # Display top 5 results
    plt.figure(figsize=(20, 10))
    for i in range(5):
        image_path, score = ls_path_score[i]
        image = plt.imread(image_path)
        plt.subplot(1, 5, i + 1)
        plt.imshow(image)

        # Extract the class name from the path
        class_name = image_path.split('/')[-2]
        plt.title(f"{class_name}")

        plt.axis('off')
    plt.suptitle("Top 5 Results")
    plt.show()

In [10]:
ROOT = 'data'
CLASS_NAME = sorted(list(os.listdir(f"{ROOT}/train"))) # Get the images' classes from data

In [ ]:
def read_image_from_path(path, size):
  img = Image.open(path).convert('RGB').resize(size) # Open the image from path, convert color to RGB type and resize the image
  return np.array(img) # Vectorize the img

def folder_to_images(folder, size):
  list_dir = [folder + '/' + name for name in os.listdir(folder)] # Get the images' path
  images = np.zeros(shape = (len(list_dir), *size, 3))
  images_path = []

  for i, path in enumerate(list_dir): # This step is to check if an image could be opened

    images[i] = read_image_from_path(path, size)
    images_path.append(path)

  return images, images_path

In [5]:
embedding_function = OpenCLIPEmbeddingFunction()

def get_single_image_embedding(img):
  embedding = embedding_function._encode_image(img = img) # Get features from image as a vector
  return np.array(embedding)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [11]:
def abs_diff(query, data):
  axis_batch_size = tuple(range(1, len(data.shape)))
  return np.sum(np.abs(data - query), axis = axis_batch_size) # Get the absolute difference of every images in data to the query image

def get_l1_score(root_img_path, query_path, size):
  query = read_image_from_path(query_path, size)
  query_embedding = get_single_image_embedding(query)
  ls_path_score = []
  for folder in os.listdir(root_img_path):
    if folder in CLASS_NAME:
      path = root_img_path + folder
      images_np, images_path = folder_to_images(path, size)
      embedding_list = []

  for idx_img in range(images_np.shape[0]):
    embedding = get_single_image_embedding(images_np[idx_img].astype(np.uint8))
    embedding_list.append(embedding)  # Add embedding to the list of embeddings for the current folder
    rates = abs_diff(query_embedding, np.stack(embedding_list))
    ls_path_score.extend(list(zip(images_path, rates)))
  return query, ls_path_score

In [14]:
def mean_square_diff(query, data):
  axis_batch_size = tuple(range(1, len(data.shape)))
  return np.mean((data - query)**2, axis = axis_batch_size)

def get_l2_score(root_img_path, query_path, size):
  query = read_image_from_path(query_path, size)
  query_embedding = get_single_image_embedding(query)
  ls_path_score = []
  for folder in os.listdir(root_img_path):
    if folder in CLASS_NAME:
      path = root_img_path + folder
      images_np, images_path = folder_to_images(path, size)
      embedding_list = []

  for idx_img in range(images_np.shape[0]):
    embedding = get_single_image_embedding(images_np[idx_img].astype(np.uint8))
    embedding_list.append(embedding) # Add embedding to the list of embeddings for the current folder
    rates = mean_square_diff(query_embedding, np.stack(embedding_list))
    ls_path_score.extend(list(zip(images_path, rates)))
  return query, ls_path_score

In [ ]:
def cosine_similarity(query, data):
  axis_batch_size = tuple(range(1,len(data.shape)))
  query_norm = np.sqrt(np.sum(query**2))
  data_norm = np.sqrt(np.sum(data**2, axis=axis_batch_size))
  return np.sum(data * query, axis=axis_batch_size) / (query_norm*data_norm + np.finfo(float).eps)

def get_CosineSimilarity_score(root_img_path, query_path, size):
  query = read_image_from_path(query_path, size)
  query_embedding = get_single_image_embedding(query)
  ls_path_score = []
  for folder in os.listdir(root_img_path):
    if folder in CLASS_NAME:
      path = root_img_path + folder
      images_np, images_path = folder_to_images(path, size)
      embedding_list = []

  for idx_img in range(images_np.shape[0]):
    embedding = get_single_image_embedding(images_np[idx_img].astype(np.uint8))
    embedding_list.append(embedding) # Add embedding to the list of embeddings for the current folder
    rates = cosine_similarity(query_embedding, np.stack(embedding_list))
    ls_path_score.extend(list(zip(images_path, rates)))
  return query, ls_path_score

In [ ]:
def correlation_coefficient(query, data):
  axis_batch_size = tuple(range(1,len(data.shape)))
  query_mean = query - np.mean(query)
  data_mean = data - np.mean(data, axis=axis_batch_size, keepdims=True)
  query_norm = np.sqrt(np.sum(query_mean**2))
  data_norm = np.sqrt(np.sum(data_mean**2, axis=axis_batch_size))

  return np.sum(data_mean * query_mean, axis=axis_batch_size) / (query_norm*data_norm + np.finfo(float).eps)

def get_CorrCoef_score(root_img_path, query_path, size):
  query = read_image_from_path(query_path, size)
  query_embedding = get_single_image_embedding(query)
  ls_path_score = []
  for folder in os.listdir(root_img_path):
    if folder in CLASS_NAME:
      path = root_img_path + folder
      images_np, images_path = folder_to_images(path, size)
      embedding_list = []

  for idx_img in range(images_np.shape[0]):
    embedding = get_single_image_embedding(images_np[idx_img].astype(np.uint8))
    embedding_list.append(embedding) # Add embedding to the list of embeddings for the current folder
    rates = correlation_coefficient(query_embedding, np.stack(embedding_list))
    ls_path_score.extend(list(zip(images_path, rates)))
  return query, ls_path_score